In [2]:
from bs4 import BeautifulSoup as soup
import requests
import numpy as np
import time
import pandas as pd
from splinter import Browser
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()
#using selenium to interact with the website
url = 'https://commerce-app.gov.in/eidb/ecntcomq.asp' #Given url
executable_path = {"executable_path": "/Users/kowshikbokka/Downloads/chromedriver"}
options = Options()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.page_load_strategy = 'eager'
driver = webdriver.Chrome(**executable_path,options=options)
driver.get(url)
page_soup_home = soup(driver.page_source, 'html.parser')
no_of_countries = len(page_soup_home.body.tbody.find_all("tr")[1].find_all("td")[1].p.find_all("option"))
no_of_years = len(page_soup_home.body.tbody.find("tr").find_all("td")[1].find_all("option"))
df = pd.DataFrame(columns = ["S.No.","HSCode","Commodity","Value_of_Export","Country","Year"])
#loop start
for i in range(1,no_of_years):
    page_soup = soup(driver.page_source, 'html.parser')
    container = page_soup.find_all("table")
    drpyear = Select(driver.find_element_by_id("select2"))
    drpyear.select_by_index(i)
    for m in range(no_of_countries):
        drpcountry = Select(driver.find_element_by_id("select3"))
        drpcountry.select_by_index(m) #country loop
        drphs = Select(driver.find_element_by_id("hslevel"))
        drphs.select_by_index(3)
        driver.find_element_by_id("radioDAll").click()
        driver.find_element_by_id("button1").click()
        page_soup = soup(driver.page_source, 'html.parser')
        try:
            df_Country = pd.read_html(driver.page_source)[0]
            df_Country.drop(df_Country.tail(3).index,inplace=True)
            df_Country['Country'] = page_soup.find_all("body")[0].find_all("font")[2].text[10:]
            df_Country.drop(['%Growth',list(df_Country.columns)[3]],axis=1,inplace=True)
            df_Country['Year'] = list(df_Country.columns)[3]
            df_Country.rename(columns={list(df_Country.columns)[3]:'Value_of_Export'},inplace=True)
            df = pd.concat([df,df_Country])
        except ValueError:
            pass 
        driver.implicitly_wait(10)
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        driver.back()


20
21
22
23


In [3]:
df.to_csv("Export Data 2")

In [4]:
df = pd.read_csv("Export Data 1")

AttributeError: 'DataFrame' object has no attribute 'read_csv'